# Projectile motion

In [1]:
# Import some libraries from python and set some defaults
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.image as mpimg
import sys
import os
import time
sys.path.append(os.getcwd())
# from scipy.interpolate import CubicSpline
import pickle
import copy
plt.rcParams.update({'font.size': 17})
params = {'axes.labelsize': 16, 'axes.titlesize': 16}
plt.rcParams.update(params)
rc('animation', html='html5')

In [2]:
# get the base file
import requests
from pathlib import Path
if Path("DEq_Solver.py").is_file():
    print("DEq_Solver already exist")
else:
    print("Downloading the DEq_solver")
    request = requests.get("https://raw.githubusercontent.com/keithonpy/Scientific_Computing/main/Base/DEq_Solver.py")
    with open("DEq_Solver.py", "wb") as f:
        f.write(request.content)
print("Deq_Solver imported")
from DEq_Solver import DEq_Solver

DEq_Solver already exist
Deq_Solver imported


Implement the second and fourth order Runge-Kutta method in the cell below. You have access to 
- the current coordinates array as ``self.x``
- the current time as ``self.t``
- the derivative $dx/dt$ as ``self.kernel.dx_dt(self.x,self.t)``
- the timestep $\Delta t$ as ``self.delta_t``.

Note that both classes below are derived from the DEq_Solver class which is defined in DEq_Solver.py (same directory). They will thus feature all methods defined in DEq_Solver as well as the methods defined for the individual derived classes.


**2nd order Runge-Kutta method**

Remeber in Radioactive decay notebook, $dx/dt = f(x, t)$, the 2nd order Runge-Kuuta method improve the Euler's method:
$$
x' = x + f(x, t)\frac{\Delta t}{2} \\
x(t + \Delta t) = x(t) + f(x', t') \Delta t \\
x^{n+1} = x^{n} + f(x^{n} + \frac{\Delta t}{2} f(x^{n}, t^{n}), t^{n}+\frac{\Delta t}{2}) \Delta t
$$

**4th order Runge-Kutta method**
Further improvement of 2nd order Runge-Kutta method by taking more points
$$
x(t + \Delta t) = x(t) + \frac{\Delta t}{6} [f(x_{1}', t_{1}') + 2f(x_{2}',t_{2}')+ 2f(x_{3}', t_{3}')+f(x_{4}', t_{4}')]
$$

where
$$
x_{1}' = x \text{   &   } t_{1}' = t 
$$
$$
x_{2}' = x + f(x_{1}', t_{1}') \text{   &   } t_{2}' = t + \frac{\Delta t}{2} \\
$$
$$
x_{3}' = x + f(x_{2}', t_{2}') \text{   &   } t_{3}' = t + \frac{\Delta t}{2} \\
$$
$$
x_{4}' = x + f(x_{3}', t_{3}') \text{   &   } t_{4} = t + \Delta t
$$


In [3]:
class RK2Solver(DEq_Solver):
    def __init__(self,kernel):
        self.kernel  = kernel
    def makeStep(self):
        # implement the method for performing a step according to the second-order Runge-Kutta algorithm
        # You have access to
        # self.t: current time
        # self.delta_t: the timestep
        # self.x: state of the system at time t
        # self.kernel.dx_dt(x, t): the derivative, dx/dt, at time t
        # input: none
        # output: none
        # the routine should update x and t
        # YOUR CODE HERE
        # inner function
        x_pi = self.x + self.kernel.dx_dt(self.x, self.t)* self.delta_t / 2
        t_pi = self.t + self.delta_t / 2
        # outer function
        self.x += self.kernel.dx_dt(x_pi, t_pi)*self.delta_t 
        self.t += self.delta_t
        
class RK4Solver(DEq_Solver):
    def __init__(self,kernel):
        self.kernel  = kernel
    def makeStep(self):
        # implement the method for performing a step according to the fourth-order Runge-Kutta algorithm
        # You have access to
        # self.t: current time
        # self.delta_t: the timestep
        # self.x: state of the system at time t
        # self.kernel.dx_dt(x, t): the derivative, dx/dt, at time t
        # input: none
        # output: none
        # the routine should update x and t
        # YOUR CODE HERE
        x1 = self.x
        t1 = self.t
        x2 = self.x + self.kernel.dx_dt(x1, t1)*self.delta_t /2
        t2 = self.t + self.delta_t /2
        x3 = self.x + self.kernel.dx_dt(x2, t2)*self.delta_t/2
        t3 = self.t + self.delta_t /2
        x4 = self.x + self.kernel.dx_dt(x3, t3)*self.delta_t
        t4 = self.t + self.delta_t
        
        self.x += self.delta_t / 6 * (self.kernel.dx_dt(x1,t1)
                                      + 2*self.kernel.dx_dt(x2, t2) 
                                      + 2*self.kernel.dx_dt(x3, t3)
                                      + self.kernel.dx_dt(x4, t4))
        self.t += self.delta_t

In [4]:
class testKernel:
    def dx_dt(x,t):
        return 2.0*t+x

solver=RK2Solver(testKernel)
x0=np.array([0.0])
solver.initialise(x0,0,1,0.1)
for i in range(5):
    solver.makeStep()
assert np.isclose(solver.x[0],0.2948935)

In [5]:
class testKernel:
    def dx_dt(x,t):
        return 2.0*t+x[1]-3.0*x[0]

solver=RK4Solver(testKernel)
x0=np.array([1.0,0.2])
solver.initialise(x0,0,1,0.1)
for i in range(5):
    solver.makeStep()
assert np.isclose(solver.x,np.array([0.29898195, -0.50101805])).all()

In projectile motion, we often want to compute the projectile's range  -- that is the distance it travels horizontally before it hits the ground (assuming Earth is flat!). To do so, we interpolate the trajectory between the two final steps (one above, and one *below* ground level) to determine the range more accurately. The function in the cell below does so by implementing **linear interpolation**.

Interpolate the value of $f(x)$ given *left*=$(x0,y0)$ and *right*=$(x1,y1)$ anchor points (where *left* is the last step 
with $y>0$, and *right* is the first step where $y<0$. $y$ is of course the height above the ground.)


In [6]:
def interpolate(x,left,right):
    # Implement linear interpolation
    # input: left:    point with Cartesian coordinates (x0, y0) = left
    #        right:   point with Cartesian coordinates (x1, y1) = right
    #        x:       x-coordinate of a point
    # output: y
    #     such that the point with Cartesian coordinates (x, y) lies on the straight line
    #     that passes through the points (x0, y0) and (x1, y1)
    x0, y0 = left
    x1, y1 = right
    # YOUR CODE HERE
    
    slope = (y1-y0) / (x1-x0)
    intercept = (x1*y0 - x0*y1) / (x1-x0)
    return slope*x+intercept

Below cell tests the implementation of linear interpolation

In [7]:
left=(0.4,4.5)
right=(0.6,7.78)
assert np.isclose(interpolate(0.45,left,right),5.32)

A second order differential equation (here, describing the trajectory of a projectile) can be re-written as two coupled, first order equations. Since we have two, second-order DEs (one for horizontal motion, one for vertical motion), we will have 4 coupled first order equations for ballistic motion in two dimensions.

We implement this by using the array ``x = np.array([s,h,vs,vh])``, where ``s`` and ``vs`` are horizontal distance and horizontal velocity, and ``h`` and ``vh`` are height and vertical velocity, respectively.

We will use the linear interpolation function to determine the range of the projectile by interpolating the last two steps of the solver to height $h = 0$.

Implement the kernel ``dx_dt`` in the class below and also complete the method for drag force (a.k.a. air resistance) as a function of velocity ${\bf v}$, 

${\bf F}_{\rm drag}(h) = -B_{2,\rm drag}(h)\,v^2\frac{\bf v}{v}$

Here ${\bf v}$ is the velocity, with Cartesian components $(vs,vh)$, and $v$ is the *speed* (magnitude of ${\bf v}$),
$v=\left(vs^2+vh^2\right)^{1/2}$.

Implement 3 different air models for the drag coefficient as a function of height, $h$:

 - constant: $B_{2,  \rm drag}=1$
 
 - isothermal: $B_{2,\rm drag}(h)=\exp(-h/10^4{\rm m})$
 
 - adiabatic: 
     - $B_{2,\rm drag}(h)=0$ for $h> (300./0.0065){\rm m}$
     
     - $B_{2,\rm drag}(h) = (1-0.0065*h/ (300{\rm m}))^{2.5}$ for $h\le (300./0.0065){\rm m}$
             
Assume the projectile's mass, is $m=1$ kg. In this expression, the factors 300 and 0.0065 apply to motion in the Earth's atmosphere.


In [ ]:
class Cannonball:
    def __init__(self,B_m,airmodel='const'):
        """Initialize a cannonball object with drag coefficient B/m
        and an optional air density model"""
        self.B_m = float(B_m)
        self.airmodel = airmodel
        
    def correctionFactor(self,height):
        # Implement the calculation of the drag coefficient, B_(2, drag)
        #   as a function of height
        # input: the height, h, in meters
        # returns: B_(2, drag)
        # As an example: constant airmodel
        if self.airmodel == 'const':  
            return 1.0
        elif self.airmodel == 'isothermal': 
        # YOUR CODE HERE
            return np.exp(-height/10**4)
        elif self.airmodel == 'adiabatic':  
        # YOUR CODE HERE
            
        else:
            raise Exception('Unknown air model',self.airmodel)

    def dx_dt(self,x,t):
        # implement the rate of change, dx/dt, including gravity and drag
        # input: t: current ime
        #        x: the location of the cannon ball at time t
        #          where x[0] is the horizontal position
        #                x[1] is the vertical position (height)
        #                x[2] is the horizontal velocity
        #                x[3] is the vertical velocity
        # return: dx/dt
        # YOUR CODE HERE




def cannon_range(solver):
    # Calculate the range of the shot, by interpolating
    # between the last two points in the solver history for which
    # the penultimate step has height h>0, and the last step has h<0.
    # Use linear interpolation to compute the range, i.e. where h=0
    # input:   the solver
    # returns: the value of the horizontal position along the trajectory where the height, h=0
    # Solution:
    # extract the (x,y) coordinates of all the steps from the solver
    xs, ys, _, _ = solver.coordinateSteps()

    # now interpolate linearly between the last two
    # YOUR CODE HERE